In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt
from mle_toolbox import load_result_logs, visualize
from neuroevobench.metrics import plot_rliable

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


envpool not installed, Atari problems will not work.
envpool not installed, Atari problems will not work.
You need to install brax for Brax tasks:
  pip install git+https://github.com/google/brax.git@main


### Load Hyper & Meta-Log

In [29]:
score_dict = {
    "OpenAI-ES": np.array([[0], [1], [2], [3]]),
    "PGPE": np.array([[0], [1], [2], [3]]) + 1,
    "ARS": np.array([[0], [1], [2], [3]]) + 2,
    "SNES": np.array([[0], [1], [2], [3]]) + 3,
    "Sep-CMA-ES": np.array([[0], [1], [2], [3]]) + 4,
    "LES": np.array([[0], [1], [2], [3]]) + 5,
    "Gaussian GA": np.array([[0], [1], [2], [3]]) + 6,
    "SAMR-GA": np.array([[0], [1], [2], [3]]) + 7,
    "GESMR-GA": np.array([[0], [1], [2], [3]]) + 8,
    "LGA": np.array([[0], [1], [2], [3]]) + 9,
}


In [31]:
plot_rliable(score_dict, "iqm", reps=100)

(<Figure size 3600x6000 with 1 Axes>,
 <Axes: title={'center': 'Interquartile Mean (IQM)'}, xlabel='Normalized Performance'>)